# Libreta para la descarga de datos del dispositivo 

## Español

Para poder ejecutar esta libreta se debe tener git instalado y la libreria **iertools**.

Para instalar git basta descargar el instalable de [esta página](https://git-scm.com/).

Puede instalar la librería en Jupyter notebook usando esta orden una vez:

```
pip install git+https://github.com/AltamarMx/iertools.git

```
Es necesario descargar el archivo xlsx del formulario y guardarlo con el nombre **Formulario.xlsx** en la carpeta **data**.

Para hacer referencia al dispositivo en Thingsboard, en el archivo **config.ini** deberás incluir los siguientes datos: 

    Dispositivo  -->    Nombre para referirte al dispositivo
    token        -->    Token del dispositivo
    device_id    -->    ID del dispositivo
    tenant       -->    Cuenta de thigsboard
    password     -->    Clave de la cuenta
    host         -->    Servidor de thingsboard
    port         -->    Puerto


---

# Notebook for downloading data from the device

## English

In order to run this notebook you must have git installed and the **iertools** library.

To install git just download the installer from [this page](https://git-scm.com/).

You can install the library on Jupyter notebook using this command once:

```
pip install git+https://github.com/AltamarMx/iertools.git

```

You need to download the xlsx file of the form and save it with the name **Formulario.xlsx** in the **data** folder.

To reference the device in Thingsboard, the following data must be included in the **config.ini** file:

    Device       -->    Name to refer to the device
    token        -->    Device Token
    device_id    -->    Device ID
    tenant       -->    Thingsboard account
    password     -->    Account password
    host         -->    Thingsboard server
    port         -->    Port

--------------------------------------------------------------------------------

In [ ]:
from iertools.tb import *
import matplotlib.pyplot as plt
import datetime
import pandas as pd

-----------------------------------------------------------------------------
En la siguiente celda llena los datos requeridos. / In the following cell fill the required data.

In [ ]:
formulario = '../data/Formulario.xlsx'             # ruta de ubicación del formulario / form path
nombre_dispositivo = 'Nombre del dispositivo'      # Nombre del dispositivo en config.ini para descargar datos de Thingsboard / Device name in config.ini for downloading data from Thingsboard

# Rango de fechas para la descarga de datos / Date range for data download
fecha1 = parse("2024-01-01")  
fecha2 = datetime.datetime.now()

-------------------------------------------------------------------------

In [ ]:
# Descarga de los datos de Thingsboard / Downloading data from Thingsboard

tmp= TB(config_file='config.ini',device_name=nombre_dispositivo) 
encuesta = [
    "Temperatura",
    "Clo",
    "Met",
    "Ubicacion",
    "Sensacion",
    "Aceptacion",
    "BPMs"
]

Dispositivo = pd.concat([tmp.get_df(key=sensor,start_datetime=fecha1,end_datetime=fecha2) for sensor in encuesta],axis=1)

In [ ]:
# Lectura de los datos del formulario / Reading data from the form

form = pd.read_excel(formulario)
form.set_index('Nombre del dispositivo',inplace=True)

In [ ]:
form.loc[nombre_dispositivo]  

In [ ]:
fecha_nac = form.loc[nombre_dispositivo]["Indique su fecha de nacimiento"]
hoy = pd.to_datetime('today')
edad = hoy.year - fecha_nac.year - ((hoy.month, hoy.day) < (fecha_nac.month, fecha_nac.day))
peso = form.loc[nombre_dispositivo]["¿Cuál es su peso? [kg]"]
altura = form.loc[nombre_dispositivo]["¿Cuál es su altura? [m]"]
sexo = form.loc[nombre_dispositivo]["¿Cuál es su sexo?"]
frecuencia = form.loc[nombre_dispositivo]["¿Con que frecuencia a la semana sueles estar en espacios con aire acondicionado?"]

In [ ]:
Dispositivo["age"] = edad
Dispositivo["weight"] = peso
Dispositivo["height"] = altura
Dispositivo["sex"] = sexo
Dispositivo["fr"] = frecuencia

In [ ]:
# Diccionarios para la presentación de la tabla / Dictionaries for the table presentation

diccionario_aceptacion = {
    0: "No",
    1: "Sí"
}

diccionario_ubicacion = {
    0: "Aula 1",
    1: "Aula 2",
    2: "Aula 3",
    3: "Aula 4",
    4: "Aula 5",
    5: "Aula 6",
    6: "Aula 7",
    7: "Aula 8",
    8: "Aula 9",
    9: "Aula 10"
}

diccionario_sexo = {
    "Masculino": "M",
    "Femenino": "F"
}

diccionario_frecuencia = {
    "Todos los días": 3,
    "3 a 5 días a la semana": 2,
    "1 a 3 días a la semana": 1,
    "Nunca": 0
}


In [ ]:
Dispositivo['Ubicacion'] = Dispositivo['Ubicacion'].map(diccionario_ubicacion)
Dispositivo['Aceptacion'] = Dispositivo['Aceptacion'].map(diccionario_aceptacion)
Dispositivo['sex'] = Dispositivo['sex'].map(diccionario_sexo)
Dispositivo['fr'] = Dispositivo['fr'].map(diccionario_frecuencia)

In [ ]:
Dispositivo.index = Dispositivo.index.strftime('%Y-%m-%d %H:%M')

In [ ]:
# Nuevos nombres de columnas para el formato de la tabla / New column names for the table formatting

nuevos_nombres = [ 'T_w', 'CLO', 'MET', 'Location', 'TS', 'A', 'BPMs', 'Age', 'W', 'H', 'Sex', 'fr' ]
Dispositivo.index.name = 'date'


Dispositivo.columns = nuevos_nombres

In [ ]:
# Visualización de la tabla / Table visualization

Dispositivo